In [36]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
%matplotlib inline
plt.ioff()

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('US_births_2000-2014_SSA.csv')

In [3]:
df

,year,month,date_of_month,day_of_week,births
0,2000,1,1,6,9083
1,2000,1,2,7,8006
2,2000,1,3,1,11363
3,2000,1,4,2,13032
4,2000,1,5,3,12558
...,...,...,...,...,...
5474,2014,12,27,6,8656
5475,2014,12,28,7,7724
5476,2014,12,29,1,12811
5477,2014,12,30,2,13634


# 1. Visualization

In [34]:
mean_births = df.births.sum() / len(df)

rel_freq = (
  df.groupby(['month', 'date_of_month'])
    .births
    .mean()
    .unstack(-1)
) / mean_births

In [ ]:
%%capture
fig, ax = plt.subplots(figsize = (10,5))
sns.heatmap(z_table, center = 0, vmin = -5, vmax = 5, annot = True,
            fmt = '.1f', square = True, cmap = 'coolwarm', cbar = False, ax = ax)

ax.set_xlabel('Beer Type')
ax.set_ylabel('Borough')
ax.set_yticklabels(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island'])
ax.set_title('Z Scores for Beer Types in Each County')


In [ ]:
fig, ax = plt.subplots(figsize = (31, 12))
sns.heatmap(rel_freq,
            ax = ax, 
            center = 1, 
            vmin = 0.5,
            vmax = 1.5,
            annot = True,
            fmt = '.2f',
            square = True,
            cmap = 'coolwarm',
            cbar = False,
            annot_kws = {'fontdict' : {'fontsize' : 15}}
           )

font_dict = {'fontsize' : 20}
ax.set_facecolor('white')
ax.set_xlabel('Day of Month', font_dict)
ax.set_ylabel('Month', font_dict)
ax.set_yticklabels([
  'January',
  'February',
  'March',
  'April',
  'May',
  'June',
  'July',
  'August',
  'September',
  'October',
  'November',
  'December'
], font_dict, rotation = 0)
ax.set_title('Relative Popularity of Birthdays, 2000–2014', fontdict = {'fontsize' : 40})#{'fontsize' : 40})
plt.xticks(fontsize = 20)
plt.yticks(rotation = 0)
fig.savefig('outputs/chart.png', transparent = True)
plt.show()